In [1]:
import warnings
import h5py
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
import numpy as np
import pandas as pd 

from nilearn import datasets, image
from nilearn import surface
from nilearn import plotting
from nilearn import input_data

from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
from nibabel.affines import apply_affine
import nibabel as nib
import time

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 
import scipy.io as sio
from scipy import stats
from numpy.linalg import inv
from numpy import inf
from scipy import stats

# import own functions
import utils
reload(utils)

sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})
sns.set(palette="colorblind")

# 1. Specify path to load masks etc.

In [2]:
path = '../fMRI_analysis_jiajia/data/' # @Jiajia put the path here

# mask_nii is the functional mask, this selects the brain voxels
mask_nii = nib.load(os.path.join(path, 'mask.nii')) 
# this where we plot our mask ON (sometimes called brain_nii) - the anatomical/structural image
mean_nii = nib.load(os.path.join(path, 'mean.nii')) 

# inverse of the affine matrix: mni2cor
inv_affine = inv(mask_nii.affine) # get the transformation matrix

# load mask and get voxel coordinates
mask_arr = np.load(path+'mask_arr.npy') # all masks are the same
mask_mat = mask_arr[0] # so we can pick any one from the array
coords_mat = np.array(np.where(mask_mat == 1)) # so need one set of voxel coordinates for all
coords_mat[[0, 2]] = coords_mat[[2, 0]] # exchange the rows

## @Jiajia adapt this to get the residuals for each subject in `residuals`

> Sanity check: `residuals.shape` should be (1698, 220075, 8) - (TRs, voxels, subjects)

In [3]:
residuals = [] # for all subjects 

# change filename to subject #
data_dir = '/ncf/gershman/Lab/scripts/matlab/VGDL_fMRI/mat/' # @Jiajia put directory of folder in which the residuals are here

num_subjects = 8

for i in range(num_subjects):
    print(i)
    idx = i+1

    filename = 'residuals_glm9_subjk_smooth.mat' # update filename with subject number
    filename = filename.replace('k', str(idx))
    
    data = h5py.File(data_dir + filename,'r')
    print(f'Get residuals for subject {idx}')
    # take the residuals from .mat file
    residuals_sub_k = data['R'].value
    
    # append to list
    residuals.append(residuals_sub_k)

0
Get residuals for subject 1
1
Get residuals for subject 2
2
Get residuals for subject 3
3
Get residuals for subject 4
4
Get residuals for subject 5
5
Get residuals for subject 6
6
Get residuals for subject 7
7
Get residuals for subject 8


## Swap axes if needed @jiajia

In [14]:
residuals = np.swapaxes(residuals,0,2)
residuals.shape

(1698, 220075, 8)

# 2. Pick seeds @jiajia just run this :)

- We loaded the whole-brain residuals
- Now, we pick ROIs (== seed voxel) and correlate their activity with other voxels in the brain
- If we find voxels that are correlated with a seed ROI, this suggests that these voxels are functionally connected

In [5]:
# Theory ENCODING voxels
R_IFG_Tri_E = [42, 28, 26]
L_Insula_E = [-30, 28, 2]
R_DMPFC_E = [6, 38, 40]
L_IFG_Tri_E = [-50, 44, 12]
L_MTG_E = [-64, -50, 4]
R_MTG_E = [58, -36, 8]
Roi_1A = [48, 34,  8]

# Theory UPDATING voxels
R_IFG_Oper_U = [48, 12, 28]
L_PPC_U = [-56, -32, 46]
R_IFG_Tri_U = [52, 38, 16]
R_AG_U = [32, -60, 34]
L_Fusiform_U = [-40, -58, -12]
L_IFG_Oper_U = [-42, 4, 28]
R_PHC_U = [26, -42, -8]

# control voxels
# tip: can always try the contralateral ROIs: [-x y z]
L_lingual = [2, -86, 4]
Occipital = [-36 -88 -12]

In [6]:
# create dictionary to map points to roi names
encoding_roi_dict = {'R_IFG_Tri_E':R_IFG_Tri_E, 'L_Insula_E':L_Insula_E, 'R_DMPFC_E':R_DMPFC_E,
                     'L_IFG_Tri_E':L_IFG_Tri_E, 'L_MTG_E':L_MTG_E, 'R_MTG_E':R_MTG_E, 'Roi_1A ':Roi_1A 
                    }

updating_roi_dict = {'R_IFG_Oper_U':R_IFG_Oper_U, 'L_PPC_U':L_PPC_U, 'R_IFG_Tri_U':R_IFG_Tri_U,
                     'R_AG_U':R_AG_U, 'L_Fusiform_U':L_Fusiform_U, 'L_IFG_Oper_U':L_IFG_Oper_U, 
                     'R_PHC_U':R_PHC_U
                    }

# combine in one
EU_dict = {**encoding_roi_dict, **updating_roi_dict}

# map control voxels to names
control_dict = {'control vox (left lingual)':L_lingual, 'control vox (Occipital)':Occipital}

In [7]:
# get the voxel indices for the theory encoding and theory updating regions
encoding_voxels = []
updating_voxels = []

# ENCODING ROIs
for key, value in encoding_roi_dict.items():

    coords_mni = value
    #print(coords_mni)
    
    coords_natv = apply_affine(aff=inv_affine, pts=coords_mni) # from mni2cor
    vox_num = utils.get_vox_from_coords(coords_mat, coords_natv) # corresponding voxel
    
    encoding_voxels.append(vox_num)

# UPDATING ROIs
for key, value in updating_roi_dict.items():

    coords_mni = value
    #print(coords_mni)
    
    coords_natv = apply_affine(aff=inv_affine, pts=coords_mni) # from mni2cor
    vox_num = utils.get_vox_from_coords(coords_mat, coords_natv) # corresponding voxel
    
    updating_voxels.append(vox_num)

The coordinates correspond to voxel: 146217.
The coordinates correspond to voxel: 89655.
The coordinates correspond to voxel: 175423.
The coordinates correspond to voxel: 114273.
The coordinates correspond to voxel: 91966.
The coordinates correspond to voxel: 102081.
The coordinates correspond to voxel: 104340.
The coordinates correspond to voxel: 150132.
The coordinates correspond to voxel: 184359.
The coordinates correspond to voxel: 123560.
The coordinates correspond to voxel: 160565.
The coordinates correspond to voxel: 53783.
The coordinates correspond to voxel: 149930.
The coordinates correspond to voxel: 63462.


# 3. Correlation matrix and connectome

> Correlate the theory updating voxels with the theory encoding voxels

<img src="http://drive.google.com/uc?export=view&id=1RSGslepHFghu4LpvuwmRkcwdRCEYD4DP" style="height:200px"/>


In [8]:
encoding_voxels

[146217, 89655, 175423, 114273, 91966, 102081, 104340]

In [9]:
updating_voxels

[150132, 184359, 123560, 160565, 53783, 149930, 63462]

In [10]:
def correlate_one_sub(R, encoding_voxels, updating_voxels):
    '''
    Correlates the time series between two sets of voxels.
    
    Parameters
    ----------
    
    R: the residuals for one subject
    updating_voxels: the indices of the updating voxels
    encoding_voxels: the indices of the encoding voxels
    
    Returns
    -------
    
    corr_matrix: asymmetric correlation matrix for that subject
    
    '''
    
    # select the voxels from the residuals
    updating_time_series = R[:, updating_voxels]
    encoding_time_series = R[:, encoding_voxels]

    N = len(encoding_time_series[1])
    corr_matrix = np.zeros((N,N))


    for i in range(N): # iterate through voxels (see img above)

        corr_col = []
        # pick the ith column from encoding 
        encoding_vec = encoding_time_series[:, i]

        # correlate it with each col (j) from updating time series
        for j in range(N):

            updating_vec = updating_time_series[:, j]

            # correlate column i with column j
            corr, _ = stats.pearsonr(encoding_vec, updating_vec)

            corr_col.append(round(corr,2)) 

        corr_matrix[:, i] = corr_col # insert correlations into correlation matrix
#         print(f'--- Theory encoding col {i} ---')
#         print(corr_matrix)

    return corr_matrix


# @jiajia I could not check this. `residuals` should be (1698, 220075, 8)

In [16]:
residuals.shape

(1698, 220075, 8)

In [17]:
all_corr_matrices = []

for i in range(num_subjects):
    # select residuals for one subject
    R = residuals[:, :, i] # all TRs, all voxels, one subject
    
    corr_matrix = correlate_one_sub(R, encoding_voxels, updating_voxels) # call above function
    
    # append to list
    all_corr_matrices.append(corr_matrix)

In [18]:
# @Jiajia save this and send to me :)
np.save('all_corr_matrices', all_corr_matrices)

# create symmetric matrices for connectome

In [19]:
all_corr_matrices_sym = []

all_roi_voxels = encoding_voxels + updating_voxels # combine voxel indices


for i in range(num_subjects):
    # select residuals for one subject
    R = residuals[:, :, i] # all TRs, all voxels, one subject
    
    all_roi_time_series = R[:, all_roi_voxels]
    
    df = pd.DataFrame(all_roi_time_series) # convert to df
    sym_corr_matrix = df.corr('pearson') # correlate all cols
    
    
    all_corr_matrices_sym.append(sym_corr_matrix)

In [20]:
# @Jiajia save this and send to me :)
np.save('all_corr_matrices_sym', all_corr_matrices_sym)

# @Jiajia lastly, having this would be useful

In [22]:
residuals_roi_voxels = []

all_roi_voxels = encoding_voxels + updating_voxels # combine voxel indices

for i in range(num_subjects):
    # select residuals for one subject
    R = residuals[:, :, i] # all TRs, all voxels, one subject
    
    all_roi_time_series = R[:, all_roi_voxels] # take only voxels of interest
    print(all_roi_time_series.shape) # @jiajia this should be: (1698, 14)
    
    residuals_roi_voxels.append(all_roi_time_series)

(1698, 14)
(1698, 14)
(1698, 14)
(1698, 14)
(1698, 14)
(1698, 14)
(1698, 14)
(1698, 14)


In [24]:
np.array(residuals_roi_voxels).swashape # @jiajia this should be (1698, 14, 8)

(8, 1698, 14)

In [ ]:
# @Jiajia save this and send to me :)
np.save('residuals_roi_voxels', residuals_roi_voxels)